In [1]:
import pandas as pd
import os
import json
import re
from datetime import datetime

# pandasの表示オプションを設定
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 80)

# --- ここでテストしたいファイルを指定 ---
info_dir = 'posts_info/unzipped_data_7z/info/'
# 確認したい特定のファイル名をここに記述します
target_file_name = '00_rocketgirl-1188140434601337485.info' 
# ------------------------------------

target_file_path = os.path.join(info_dir, target_file_name)

print(f"--- 🎯 テスト対象ファイル ---")
print(target_file_path)

--- 🎯 テスト対象ファイル ---
posts_info/unzipped_data_7z/info/00_rocketgirl-1188140434601337485.info


In [2]:
try:
    with open(target_file_path, 'r', encoding='utf-8') as f:
        post_data = json.load(f)
    
    print("\n--- 📖 ファイル内容（JSON） ---")
    # 読み込んだJSONデータを整形して表示
    print(json.dumps(post_data, indent=2, ensure_ascii=False))

except FileNotFoundError:
    print(f"エラー: 指定されたファイル '{target_file_path}' が見つかりません。")
except json.JSONDecodeError:
    print(f"エラー: ファイル '{target_file_name}' は有効なJSON形式ではありません。")
except Exception as e:
    print(f"エラーが発生しました: {e}")


--- 📖 ファイル内容（JSON） ---
{
  "gating_info": null,
  "viewer_can_reshare": true,
  "display_resources": [
    {
      "src": "https://scontent-lax3-1.cdninstagram.com/vp/b098887b0f8664bd90550200a792aeb3/5DAD22E3/t51.2885-15/sh0.08/e35/s640x640/12729501_1503731389933869_1705238870_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com",
      "config_width": 640,
      "config_height": 640
    },
    {
      "src": "https://scontent-lax3-1.cdninstagram.com/vp/5140537c741a94f035976275b40b6986/5DB9A7DE/t51.2885-15/sh0.08/e35/s750x750/12729501_1503731389933869_1705238870_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com",
      "config_width": 750,
      "config_height": 750
    },
    {
      "src": "https://scontent-lax3-1.cdninstagram.com/vp/0f3cb748e87da90c4f3d7fd038b663d8/5DAC1948/t51.2885-15/e35/12729501_1503731389933869_1705238870_n.jpg?_nc_ht=scontent-lax3-1.cdninstagram.com",
      "config_width": 1080,
      "config_height": 1080
    }
  ],
  "viewer_in_photo_of_you": false,
  "viewer_has_saved

In [3]:
if 'post_data' in locals():
    hashtags_list = []
    mentions_list = []

    # ファイル名から投稿者を取得
    source_user = target_file_name.split('-')[0]
    
    # タイムスタンプを取得
    timestamp = post_data.get('taken_at_timestamp', 0)
    dt_object = datetime.fromtimestamp(timestamp)
    
    # --- ハッシュタグの抽出 ---
    caption_edges = post_data.get('edge_media_to_caption', {}).get('edges', [])
    caption = caption_edges[0]['node']['text'] if caption_edges else ""
    
    found_hashtags = re.findall(r"#(\w+)", caption)
    for tag in found_hashtags:
        hashtags_list.append({
            'source': source_user,
            'target': f"#{tag}",
            'timestamp': timestamp,
            'datetime': dt_object
        })

    # --- メンションの抽出 ---
    tagged_users_edges = post_data.get('edge_media_to_tagged_user', {}).get('edges', [])
    for edge in tagged_users_edges:
        tagged_user = edge.get('node', {}).get('user', {}).get('username')
        if tagged_user:
            mentions_list.append({
                'source': source_user,
                'target': tagged_user,
                'timestamp': timestamp,
                'datetime': dt_object
            })

    print("--- ✅ 抽出処理完了 ---")
    print(f"キャプション原文:\n---\n{caption}\n---")
    
    # --- 結果表示 ---
    print("\n--- 📝 抽出されたハッシュタグ ---")
    df_hashtags = pd.DataFrame(hashtags_list)
    if not df_hashtags.empty:
        display(df_hashtags)
    else:
        print("この投稿にはハッシュタグがありませんでした。")

    print("\n" + "="*50 + "\n")

    print("--- 👥 抽出されたメンション ---")
    df_mentions = pd.DataFrame(mentions_list)
    if not df_mentions.empty:
        display(df_mentions)
    else:
        print("この投稿にはメンションがありませんでした。")

else:
    print("エラー: セル2を先に実行して、データを読み込んでください。")

--- ✅ 抽出処理完了 ---
キャプション原文:
---
Tough day. Honestly my favorite Rockets player since I became a fan. I definitely don't have more photos with any other player than DMo. He was always so kind and accessible with fans that photo opportunities were easy to come by. Met him for the first time at summer league in 2012 (center pic). I loved how he genuinely loved the game of basketball. I loved his post moves. I loved his humor. He was one i wanted to be a Rockets lifer (like a Murphy or Rudy T) and then have him be a color commentator when he retired. I'm really going to miss him. 
Best of luck to you @donatas_motiejunas in your basketball career. I hope it's long & healthy and I hope it brings you back to Houston again someday. Thanks for everything. 
#rockets #houstonrockets #rednation #donatasmotiejunas #motiejunas #dmo #bffs #dontleave
---

--- 📝 抽出されたハッシュタグ ---


,source,target,timestamp,datetime
0,00_rocketgirl,#rockets,1455857398,2016-02-19 13:49:58
1,00_rocketgirl,#houstonrockets,1455857398,2016-02-19 13:49:58
2,00_rocketgirl,#rednation,1455857398,2016-02-19 13:49:58
3,00_rocketgirl,#donatasmotiejunas,1455857398,2016-02-19 13:49:58
4,00_rocketgirl,#motiejunas,1455857398,2016-02-19 13:49:58
5,00_rocketgirl,#dmo,1455857398,2016-02-19 13:49:58
6,00_rocketgirl,#bffs,1455857398,2016-02-19 13:49:58
7,00_rocketgirl,#dontleave,1455857398,2016-02-19 13:49:58




--- 👥 抽出されたメンション ---


,source,target,timestamp,datetime
0,00_rocketgirl,donatas_motiejunas,1455857398,2016-02-19 13:49:58
